In [2]:
from tools import *

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import StratifiedKFold, cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score

# Initialize Gaussian Naïve Bayes
gnb = GaussianNB()

# K-Fold Cross Validation (Stratified)
kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

cv_scores = []
for train_idx, val_idx in kf.split(X_train, y_train):
    X_train_fold, X_val_fold = X_train.iloc[train_idx], X_train.iloc[val_idx]
    y_train_fold, y_val_fold = y_train[train_idx], y_train[val_idx]

    # Train Naïve Bayes
    model = GaussianNB()
    model.fit(X_train_fold, y_train_fold)

    # Validate model
    y_val_pred = model.predict(X_val_fold)
    acc = accuracy_score(y_val_fold, y_val_pred)
    cv_scores.append(acc)

print(f"Cross-validation accuracy: {np.mean(cv_scores):.4f} ± {np.std(cv_scores):.4f}")

zfinal_model = GaussianNB()
final_model.fit(X_train, y_train)

# Predict on test set
y_test_pred = final_model.predict(X_test)

# Convert predictions back to original labels
y_test_pred_labels = label_encoder.inverse_transform(y_test_pred)

# Save for Kaggle submission
submission = pd.DataFrame({"id": test_raw.index, "y": y_test_pred_labels})
submission.to_csv("submission_gnb.csv", index=False)

print("Submission file saved as submission_gnb.csv 🚀")


Cross-validation accuracy: 0.6500 ± 0.0087
Submission file saved as submission_gnb.csv 🚀


C:\Users\natha\AppData\Local\Temp\ipykernel_35404\2200029785.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train[bool_cols] = X_train[bool_cols].astype(int)
C:\Users\natha\AppData\Local\Temp\ipykernel_35404\2200029785.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train[col] = le.fit_transform(X_train[col])


In [5]:
import pandas as pd
import numpy as np
from sklearn.feature_selection import f_classif

# Compute ANOVA F-score for numerical features
numerical_features = train_raw.select_dtypes(include=['float64', 'int64']).columns
X_num = train_raw[numerical_features]
y = train_raw['y']

anova_f, anova_p = f_classif(X_num, y)
feature_importance = pd.DataFrame({'Feature': numerical_features, 'F-Score': anova_f, 'P-Value': anova_p})
feature_importance = feature_importance.sort_values(by="F-Score", ascending=False)

print(feature_importance)


from sklearn.feature_selection import RFE
from sklearn.naive_bayes import GaussianNB

# Fit GNB model
gnb = GaussianNB()
rfe = RFE(gnb, n_features_to_select=8)  # Adjust the number of features you want
rfe.fit(X_num, y)

# Get selected features
selected_features = X_num.columns[rfe.support_]
print(f"Selected Features: {selected_features.tolist()}")


   Feature     F-Score        P-Value
9      x11  643.360054  2.401895e-249
3       x4  641.254523  1.282248e-248
8      x10    5.667893   3.477389e-03
2       x3    3.787020   2.272811e-02
4       x5    1.624479   1.971183e-01
0       x1    1.042293   3.527219e-01
1       x2    0.779779   4.585630e-01
6       x8    0.655626   5.191617e-01
5       x6    0.475849   6.213858e-01
7       x9    0.376437   6.863216e-01
10     x13    0.170474   8.432700e-01


ValueError: when `importance_getter=='auto'`, the underlying estimator GaussianNB should have `coef_` or `feature_importances_` attribute. Either pass a fitted estimator to feature selector or call fit before calling transform.